In [2]:
import pandas as pd
import numpy as np

In [25]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

postal_codes = pd.DataFrame(tables[0])

In [26]:
postal_codes = postal_codes[postal_codes.Borough != "Not assigned"]

In [28]:
df = pd.DataFrame(postal_codes.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index())
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [45]:
df.shape

(103, 3)